# TRANSFORMATION

Importamos librerías y leemos el fichero

In [ ]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
from scipy import stats

# Ajustes de matplotlib para que los gráficos salgan de forma legible
plt.rcParams['figure.figsize'] = (10, 5)
plt.rcParams['font.size'] = 10

In [ ]:
# Leemos el CSV
df = pd.read_csv("../data/retail_store_sales.csv")

In [ ]:
# Añadimos path para cargar funciones
import sys
import os

# Añade la carpeta 'src' al path de Python
sys.path.append(os.path.abspath(os.path.join('..', 'src')))

# Ahora podemos importar funciones.py
import funciones

## 4. Limpieza

### 4.1. Corregir tipos de datos

Convertimos a datetime la variable **Transaction Date** ya que hasta ahora venía dada como tipo objeto.

In [ ]:

from funciones import convertir_a_datetime

In [ ]:
clean_df = convertir_a_datetime(df, ["Transaction Date"])
clean_df.info()

Columna 'Transaction Date' convertida a datetime. Nuevos NaT generados: 0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12575 entries, 0 to 12574
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Transaction ID    12575 non-null  object        
 1   Customer ID       12575 non-null  object        
 2   Category          12575 non-null  object        
 3   Item              11362 non-null  object        
 4   Price Per Unit    11966 non-null  float64       
 5   Quantity          11971 non-null  float64       
 6   Total Spent       11971 non-null  float64       
 7   Payment Method    12575 non-null  object        
 8   Location          12575 non-null  object        
 9   Transaction Date  12575 non-null  datetime64[ns]
 10  Discount Applied  8376 non-null   object        
 11  Calculated Total  11362 non-null  float64       
dtypes: datetime64[ns](1), float64(4), object(7)
memory usage

### 4.2. Imputación de valores faltantes de las variable Item

En el dataset original, la columna **Item** contiene valores faltantes. Al analizar la estructura de los datos, observamos que cada valor de **Item** sigue un patrón común:

Item_<número>_<prefijo_de_categoria_en_mayúsculas>


Por ejemplo:

- Item_12_FOOD

- Item_27_MILK

Esto indica que la variable **Item** no es un nombre de producto, sino un identificador compuesto. El identificador combina un número aleatorio asociado al ítem y un prefijo en mayúsculas que procede de la categoría a la que pertenece.

Por lo tanto, para rellenar los valores faltantes de **Item**, se mantiene la lógica del dataset original, generando un nuevo identificador cumpliendo el mismo estándar.

In [ ]:
from funciones import imputar_items_por_categoria

In [ ]:
suffix_map = {
    "Food": "FOOD",
    "Patisserie": "PAT",
    "Milk Products": "MILK",
    "Butchers": "BUT",
    "Beverages": "BEV",
    "Furniture": "FUR",
    "Computers and electric accessories": "CEA",
    "Electric household essentials": "EHE"
}

clean_df = imputar_items_por_categoria(df, suffix_map)


Número de valores nulos en Item después de imputación: 0

Ejemplos de filas imputadas:


,Category,Item
5,Patisserie,Item_22_PAT
7,Furniture,Item_27_FUR
11,Milk Products,Item_7_MILK
15,Beverages,Item_15_BEV
17,Milk Products,Item_12_MILK
19,Furniture,Item_26_FUR
21,Milk Products,Item_16_MILK
25,Furniture,Item_27_FUR
32,Food,Item_9_FOOD
34,Patisserie,Item_9_PAT


### 4.3. Imputación de valores faltantes de las variables Price Per Unit, Quantity y Total Spent

In [ ]:
from funciones import imputar_precios_cantidades_totales


In [ ]:
clean_df = imputar_precios_cantidades_totales(clean_df)


Valores nulos restantes por columna:
Transaction ID         0
Customer ID            0
Category               0
Item                   0
Price Per Unit         0
Quantity               0
Total Spent            0
Payment Method         0
Location               0
Transaction Date       0
Discount Applied    4199
Calculated Total    1213
dtype: int64


La columna **Price Per Unit** es fundamental para la coherencia del dataset, pues forma parte directa de la relación:

Total Spent=Price Per Unit×Quantity

Por lo tanto, vamos a aplicar una imputación jerárquica y lógica.

En primer lugar, haremos una reconstrucción matemática cuando es posible.
Si Total Spent y Quantity existen, se calcula:

Price Per Unit=Total Spent/Quantity

Este es el método más fiable porque no introduce sesgo externo y respeta la naturaleza real del dato.

Cunado no es posible reconstruir, imputaremos la media agrupada por Item.
Si también faltan los valores anteriores, se usa la media del mismo producto (Item) para mantener consistencia ya que productos idénticos suelen tener precios comparables.

Como tercera alternativa, valor por defecto = 50 cuando no hay alternativa.
Si aún no existe forma de imputar, se asigna un valor constante neutral para evitar dejar nulos en la base. Este último paso afecta a muy pocos registros y sirve como último recurso.


En cuanto a la imputación en **Quantity**, vamos a aplicar el mismo razonamiento que en la columna anterior, pues las tres variables están relacionadas mediante la misma fórmula:

Quantity=Total Spent/Price Per Unit

Como segunda opción, imputaremos la media por artículo (Item) si no es posible.

Como última opción, imputaremos un valor por defecto = 5 unidades si aún falta valor.

Este enfoque mantiene la coherencia matemática del dataset, la similaridad entre productos del mismo tipo y la eliminación definitiva de nulos



Por último, en cuanto a la imputación en **Total Spent**, una vez las columnas Price Per Unit y Quantity ya están completas, imputar los nulos restantes es trivial.

Total Spent=Price Per Unit×Quantity

Este método recupera el dato exacto con la menor incertidumbre posible.

### 4.4. Imputación de valores de la variable Discount Applied

In [ ]:
from funciones import limpiar_discount_y_columna_calculada


In [ ]:
clean_df = limpiar_discount_y_columna_calculada(clean_df)

Valores nulos restantes en cada columna:
Transaction ID      0
Customer ID         0
Category            0
Item                0
Price Per Unit      0
Quantity            0
Total Spent         0
Payment Method      0
Location            0
Transaction Date    0
Discount Applied    0
dtype: int64


La columna **Discount Applied** solo puede tomar tres valores (True, False, NaN). Al analizar la distribución observamos que la mayoría de valores no aplican descuento. Los valores nulos no indican explícitamente lo contrario.

Por lo tanto, decidimos imputar los faltantes como False, siendo la decisión más conservadora y coherente con el comportamiento mayoritario de la base.

Eliminamos la columna Calculated Total creada para los cálculos.

### 4.5. Estandarización del nombre de las columnas

Estandarizamos el nombre de las columnas antes de guardar el dataframe limpio, para que estén escritos en minúsculas y separadas las palabras por guión bajo (snake_case).

In [ ]:
def estandarizar_nombres_columnas(df, verbose=True):

    clean_df = df.copy()

    clean_df.columns = (
        clean_df.columns
            .str.lower()
            .str.replace(" ", "_", regex=False)
            .str.replace("-", "_", regex=False)
    )

    if verbose:
        print("Nombres de columnas estandarizados:")
        print(clean_df.columns.tolist())

    return clean_df

In [ ]:
clean_df = estandarizar_nombres_columnas(clean_df)


Nombres de columnas estandarizados:
['transaction_id', 'customer_id', 'category', 'item', 'price_per_unit', 'quantity', 'total_spent', 'payment_method', 'location', 'transaction_date', 'discount_applied']


### 4.6. Guardar CSV limpio

Una vez el dataframe limpio, lo guardamos.

In [ ]:
# Guardar el dataframe limpio como CSV
clean_df.to_csv("../data/retail_store_sales_clean.csv", index=False)
